In [ ]:
!pip install -U aiogram
!pip install -U aiogram openai nest_asyncio
!pip install openai
!pip install -U google-generativeai
import asyncio
import openai
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.types import Message
import os
import logging
import sys
from google.colab import userdata
import nest_asyncio

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TELEGRAM_BOT_TOKEN'] = userdata.get('TELEGRAM_BOT_TOKEN')

In [ ]:
nest_asyncio.apply()

In [ ]:
user_conversations = {}
MAX_HISTORY_LENGTH = 200

In [ ]:
dp = Dispatcher()

active_users = set()

@dp.message(Command("start"))
async def cmd_start(message: Message):
    await message.reply("Hi\nI am Your Buddy.\nDeveloped by Sushean")


@dp.message(Command("help"))
async def cmd_help(message: Message):
    help_text = (
        "🆘 *Help Menu*\n\n"
        "/start - Start the bot\n"
        "/help - Show this help message\n"
        "/StartChat - Start GPT chat mode\n"
        "/close - End GPT chat mode\n"
        "Use Imagine Keyword at start of chat to Genegate images \n"
        "Type anything else to talk to the bot (when in chat mode)!"
    )
    await message.reply(help_text, parse_mode="Markdown")

@dp.message(Command("StartChat"))
async def cmd_start_chat(message: Message):
    user_id = message.from_user.id
    active_users.add(user_id)
    user_conversations[user_id] = []  # Start fresh conversation memory
    await message.reply("✅ Chat mode activated. Send me anything and I’ll respond!\nSend /close to stop.")

@dp.message(Command("close"))
async def cmd_close_chat(message: Message):
    user_id = message.from_user.id
    active_users.discard(user_id)
    user_conversations.pop(user_id, None)  # Clear memory
    await message.reply("❌ Chat mode deactivated.")

In [ ]:
@dp.message(lambda msg: msg.text.lower().startswith("imagine"))
async def handle_imagine(message: Message):
    user_id = message.from_user.id
    if user_id in active_users:
        try:
            prompt = message.text[len("imagine"):].strip()
            if not prompt:
                await message.reply("⚠️ Please provide a description after 'imagine'.")
                return

            await message.reply("🧠 Generating image... please wait ⏳")

            response = openai.images.generate(
                model="dall-e-3",  
                prompt=prompt,
                n=1,
                size="1024x1024",  
                quality="standard"  
            )

            image_url = response.data[0].url
            await message.reply_photo(photo=image_url, caption=f"🖼️ *Here's your image for:* `{prompt}`", parse_mode="Markdown")

        except Exception as e:
            await message.reply(f"❌ Failed to generate image:\n`{str(e)}`", parse_mode="Markdown")
    else:
        await message.reply("⚠️ You need to start chat mode first using /StartChat.")


In [ ]:
@dp.message(lambda msg: msg.photo)
async def handle_photo(message: Message):
    user_id = message.from_user.id
    if user_id in active_users:
        try:
            photo = message.photo[-1]
            file = await message.bot.get_file(photo.file_id)
            file_path = file.file_path
            file_url = f"https://api.telegram.org/file/bot{os.environ['TELEGRAM_BOT_TOKEN']}/{file_path}"

            # Ensure content is always a valid string
            prompt_text = message.caption if message.caption else "Describe this image."

            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": file_url}}
                    ]}
                ],
                max_tokens=200
            )

            await message.reply(response.choices[0].message.content.strip(), parse_mode="Markdown")

        except Exception as e:
            await message.reply(f"⚠️ Error handling photo:\n{e}{file_url}")
    else:
        await message.reply("🤖 Not in chat mode.\nUse /StartChat to begin.")

@dp.message(lambda msg: msg.voice)
async def handle_voice(message: Message):
    user_id = message.from_user.id
    if user_id in active_users:
        try:
            voice = message.voice
            file = await message.bot.get_file(voice.file_id)
            file_path = file.file_path
            file_url = f"https://api.telegram.org/file/bot{os.environ['TELEGRAM_BOT_TOKEN']}/{file_path}"

            import requests
            import tempfile

            r = requests.get(file_url)
            with tempfile.NamedTemporaryFile(delete=False, suffix=".oga") as temp_audio:
                temp_audio.write(r.content)
                temp_audio.flush()

                audio_file = open(temp_audio.name, "rb")
                transcript = openai.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file
                )
                text = transcript.text

            await message.reply(f"🗣 You said: {text}")

            # Optional: GPT reply
            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": text}
                ]
            )
            await message.reply(response.choices[0].message.content.strip())

        except Exception as e:
            await message.reply(f"⚠️ Error handling audio: {e}")
    else:
        await message.reply("🤖 Not in chat mode.\nUse /StartChat to begin.")


In [ ]:
@dp.message()
async def handle_message(message: Message):
    user_id = message.from_user.id

    if user_id not in active_users:
        await message.reply("🤖 Not in chat mode.\nUse /StartChat to begin.")
        return

    if not message.text:
        await message.reply("⚠ Please send a text message.")
        return

    prompt = message.text

    try:

        history = user_conversations.get(user_id, [])
        history.append({"role": "user", "content": prompt})

        # Keep only the last N messages for memory
        if len(history) > MAX_HISTORY_LENGTH:
            history = history[-MAX_HISTORY_LENGTH:]

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "You're a helpful assistant."}] + history,
            temperature=0.7
        )

        reply = response.choices[0].message.content.strip()

        history.append({"role": "assistant", "content": reply})
        user_conversations[user_id] = history  # Save updated memory

        await message.reply(reply, parse_mode="Markdown")

    except Exception as e:
        await message.reply(f"⚠ Error while generating response.\n{e}")

In [ ]:
async def main():
  bot = Bot(os.environ['TELEGRAM_BOT_TOKEN'])
  await dp.start_polling(bot)

In [ ]:
if __name__ == "__main__":
    import asyncio
    logging.basicConfig(level=logging.INFO, stream=sys.stdout)
    asyncio.run(main())